In [14]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
import numpy as np
from sklearn.model_selection import cross_val_score
import pandas as pd

dataset = pd.read_csv('data_banknote_authentication.txt', header=None).values
money_attr = dataset[:, :4]
money_attr = money_attr.astype(np.float64)
money_class = dataset[:, -1]
money_class = money_class.astype(np.int, copy=False)

kFold=cross_validation.KFold(n=len(money_attr),n_folds=5, random_state=8, shuffle = True)
#Возьмем методы классификации  QDA и KNeighborsClassifier и посчитаем для них различные метрики 
qda=QDA()
clf = KNeighborsClassifier(15)


In [15]:
#Точность классификации (Classification Accuracy)
#metric Accuracy for KNeighborsClassifier and QDA
scores = cross_val_score(clf, money_attr, money_class,  cv=kFold, scoring='accuracy')
print("Accuracy for KNeighbors: %0.3f (%0.3f)" % (scores.mean(), scores.std() ))
scores = cross_val_score(qda, money_attr, money_class, cv=kFold, scoring='accuracy')
print("Accuracy for QDA: %0.3f (%0.3f)" % (scores.mean(), scores.std() ))


Accuracy for KNeighbors: 0.999 (0.001)
Accuracy for QDA: 0.985 (0.005)


In [16]:
#Логарифм функции правдоподобия (Logarithmic Loss)
#metric Logarithmic Loss for KNeighborsClassifier and QDA
#Интересно что на моем датасете если в кросс валидации не указывать параметр shuffle = True, neg_log_loss 
#отказывается работать, так как по всей видимости тестовая выборка оказывается заполнена только 0
scores = cross_val_score(clf,  money_attr, money_class, cv=kFold, scoring='neg_log_loss')
print("log_loss for KNeighbors: %0.3f (%0.3f)" % (scores.mean(), scores.std() ))

scores = cross_val_score(qda, money_attr,money_class,  cv=kFold, scoring='neg_log_loss')
print("log_loss for QDA: %0.3f (%0.3f)" % (scores.mean(), scores.std() ))

log_loss for KNeighbors: -0.008 (0.001)
log_loss for QDA: -0.026 (0.004)


In [17]:
#Область под кривой ошибок (Area Under ROC Curve)
#metric ROC_auc for KNeighborsClassifier and QDA
# То же самое что  присоходит с neg_log_loss, происходит и с метрикой ROC_auc, без параметра shuffle = True,
#работать не будет, причина так же самая
scores = cross_validation.cross_val_score(clf, money_attr, money_class, cv=kFold, scoring='roc_auc')
print("auc for KNeighbors: %0.3f (%0.3f)" % (scores.mean(), scores.std() ))

scores = cross_validation.cross_val_score(qda, money_attr, money_class, cv=kFold, scoring='roc_auc')
print("auc for QDA: %0.3f (%0.3f)" % (scores.mean(), scores.std() ))


auc for KNeighbors: 1.000 (0.000)
auc for QDA: 1.000 (0.000)


In [18]:
#Матрица неточностей (Confusion Matrix)
#metric confusion matrix for KNeighborsClassifier and QDA
from sklearn.metrics import confusion_matrix
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(money_attr, money_class, test_size=0.3, random_state=7)


qda.fit(X_train, Y_train)
qda_predicted = qda.predict(X_test)
qda_matrix = confusion_matrix(Y_test, qda_predicted)
print("confusion matrix for QDA")
print(qda_matrix)

clf.fit(X_train,Y_train)
clf_predicted=clf.predict(X_test)
clf_matrix=confusion_matrix(Y_test,clf_predicted)
print("confusion matrix for KNeighbors")
print(clf_matrix)


confusion matrix for QDA
[[233   7]
 [  0 172]]
confusion matrix for KNeighbors
[[240   0]
 [  0 172]]


In [20]:
#Отчет классификации (Classification Report)
#metric classification report for KNeighborsClassifier and QDA
from sklearn.metrics import classification_report
qda_report=classification_report(Y_test,qda_predicted)
print('classification report for QDA')
print(qda_report)


clf_report=classification_report(Y_test,clf_predicted)
print('classification report for KNeighbors')
print(clf_report)

classification report for QDA
             precision    recall  f1-score   support

          0       1.00      0.97      0.99       240
          1       0.96      1.00      0.98       172

avg / total       0.98      0.98      0.98       412

classification report for KNeighbors
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       240
          1       1.00      1.00      1.00       172

avg / total       1.00      1.00      1.00       412



In [21]:
#Основываясь на полученных метриках можно сделать вывод что классификаторы вполне применимы 
#для даннного датасета, так как показывают высокую точность классификации на всех метриках.